# Init

In [ ]:
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider
import os
import importlib
from argparse import Namespace
sys.path.append(str(Path.cwd().parent.parent.absolute()))

from utils.utils_generic import get_logger
logger = get_logger("composable-tasks-upgraded")

## Config

In [ ]:
os.environ["MX_DEX_ENV"] = "shadowfork4"

import config
importlib.reload(config)
from context import Context
from utils.utils_chain import WrapperAddress
from contracts.composable_tasks_contract import ComposableTasksContract
from contracts.position_creator_contract import PositionCreatorContract
from tools.chain_simulator_connector import ChainSimulator

context = Context()
composable_tasks_contract: ComposableTasksContract = context.get_contracts(config.COMPOSABLE_TASKS)[0]
position_creator_contract: PositionCreatorContract = context.get_contracts(config.POSITION_CREATOR)[0]

if config.CURRENT_ENV.value == "chainsim":
    chain_sim = ChainSimulator()

In [ ]:
print(composable_tasks_contract.address)
print(context.network_provider.proxy.get_account(WrapperAddress(composable_tasks_contract.address)).contract_code_hash.hex())

In [ ]:
print(position_creator_contract.address)
print(context.network_provider.proxy.get_account(WrapperAddress(position_creator_contract.address)).contract_code_hash.hex())

# Upgrade

In [ ]:
from time import sleep
wasm_path = "https://github.com/multiversx/mx-exchange-tools-sc/releases/download/v1.1.0-rc2/composable-tasks.wasm"
contract_code_hash = "ac0d5ff9bf4d3995889c9a2e6486cefb003371717e68156ebeb3e4c6e5f7cee0"

composable_tasks_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, wasm_path)
sleep(6)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(composable_tasks_contract.address)).contract_code_hash.hex()
assert code_hash == contract_code_hash

In [ ]:
from utils.utils_tx import endpoint_call
endpoint_call(context.network_provider.proxy, 80000000, context.deployer_account, WrapperAddress(composable_tasks_contract.address), "setSmartSwapFeePercentage", [100])

# Operation

## Setup

In [ ]:
from multiversx_sdk.abi import Abi
from utils.utils_tx import endpoint_call, multi_esdt_endpoint_call
from utils.utils_chain import Account, dec_to_padded_hex
from contracts.pair_contract import PairContract

purse_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
purse_account.address = WrapperAddress(config.SHADOWFORK_FUNDING_ADDRESS)
purse_account.sync_nonce(context.network_provider.proxy)

abi = Abi.load(config.HOME / "Projects/dex/mx-exchange-tools-sc/output-docker/composable-tasks/composable-tasks.abi.json")

def wrap_egld_task():
    return (0, [])

def unwrap_egld_task():
    return (1, [])

def swap_input_task(out_token: str, min_amount: int):
    return (2, ["swapTokensFixedInput", out_token, min_amount.to_bytes()])

def swap_output_task(out_token: str, exp_amount: int):
    return (2, ["swapTokensFixedOutput", out_token, bytes.fromhex(dec_to_padded_hex(exp_amount))])

def smart_swap_task(smart_swap_args: list):
    for i in range(len(smart_swap_args)):
        element = smart_swap_args[i]
        if isinstance(element, int):
            smart_swap_args[i] = bytes.fromhex(dec_to_padded_hex(element))
        elif isinstance(element, str):
            if element.startswith("erd"):
                smart_swap_args[i] = WrapperAddress(element).get_public_key()
            else:
                pass
        elif isinstance(element, bytes):
            pass
        else:
            raise ValueError(f"Unhandled element type: {type(element)}")
        
    return (5, smart_swap_args)

In [ ]:
position_creator_abi = Abi.load(config.HOME / "Projects/dex/mx-exchange-tools-sc/output-docker/auto-pos-creator/auto-pos-creator.abi.json")

def pos_creator_route(route_args: list):
    for i in range(len(route_args)):
        element = route_args[i]
        if isinstance(element, int):
            route_args[i] = bytes.fromhex(dec_to_padded_hex(element))
        elif isinstance(element, str):
            if element.startswith("erd"):
                route_args[i] = WrapperAddress(element).get_public_key()
            else:
                pass
        elif isinstance(element, bytes):
            pass
        else:
            raise ValueError(f"Unhandled element type: {type(element)}")
        
    return route_args

## Executables

Wrap EGLD

In [ ]:
pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
print(pair_contract.get_config_dict())

args = [
    (wegld_mex_contract.firstToken, 0, 1 * 10 ** 18),
    [wrap_egld_task()]
]
endpoint_call(context.network_provider.proxy, 80000000, user, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, value=100 * 10 ** 18, abi = abi)

Simple swap

In [ ]:
pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]

args = [
    (pair_contract.secondToken, 0, 1),
    [
        wrap_egld_task(),
        swap_input_task(pair_contract.secondToken, 1)
    ]
]
endpoint_call(context.network_provider.proxy, 80000000, purse_account, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, value=1 * 10 ** 18, abi = abi)

Smart swaps

In [ ]:
from utils.utils_chain import dec_to_padded_hex
pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]

swaps_unencoded = [
    1,  # number of operations
    1 * 10 ** 18,  # amount for first operation
    3,  # number of swaps per first operation
    pair_contract.address, "swapTokensFixedInput", pair_contract.secondToken, 1,  # swap 1
    pair_contract.address, "swapTokensFixedInput", pair_contract.firstToken, 1,  # swap 2
    pair_contract.address, "swapTokensFixedInput", pair_contract.secondToken, 1,  # swap 3
]

args = [
    (pair_contract.secondToken, 0, 1),
    [
        wrap_egld_task(),
        smart_swap_task(swaps_unencoded)
    ]
]
endpoint_call(context.network_provider.proxy, 80000000, context.deployer_account, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, value=1 * 10 ** 18, abi = abi)

In [ ]:
from utils.utils_chain import dec_to_padded_hex
pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[1]

swaps_unencoded = [
    1,  # number of operations
    1 * 10 ** 18,  # amount for first operation
    3,  # number of swaps per first operation
    pair_contract.address, "swapTokensFixedInput", pair_contract.secondToken, 1,  # swap 1
    pair_contract.address, "swapTokensFixedInput", pair_contract.secondToken, 1,  # swap 2
    pair_contract.address, "swapTokensFixedInput", pair_contract.secondToken, 1,  # swap 3
]

args = [
    (pair_contract.secondToken, 0, 1),
    [
        wrap_egld_task(),
        smart_swap_task(swaps_unencoded)
    ]
]
endpoint_call(context.network_provider.proxy, 80000000, context.deployer_account, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, value=1 * 10 ** 18, abi = abi)

consistent swaps length

In [ ]:
user = Account(pem_file="~/Projects/dev/sh1.pem")
user.sync_nonce(context.network_provider.proxy)

In [ ]:
wegld_mex_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
wegld_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[1]
mex_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[-3]
print(f"{wegld_mex_contract.firstToken} {wegld_mex_contract.secondToken}")
print(f"{wegld_usdc_contract.firstToken} {wegld_usdc_contract.secondToken}")
print(f"{mex_usdc_contract.firstToken} {mex_usdc_contract.secondToken}")

In [ ]:
wegld_mex_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
wegld_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[1]
mex_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[-3]

swaps_unencoded = [
    2,  # number of operations
    5 * 10 ** 17,  # amount for first operation
    1,  # number of swaps per first operation
    wegld_usdc_contract.address, "swapTokensFixedInput", wegld_usdc_contract.secondToken, 1,  # swap 1
    5 * 10 ** 17,  # amount for second operation
    2,  # number of swaps per second operation
    wegld_mex_contract.address, "swapTokensFixedInput", wegld_mex_contract.secondToken, 1,  # swap 1
    mex_usdc_contract.address, "swapTokensFixedInput", mex_usdc_contract.secondToken, 1,  # swap 2
]

args = [
    (wegld_usdc_contract.secondToken, 0, 1),
    [
        wrap_egld_task(),
        smart_swap_task(swaps_unencoded),
    ]
]
endpoint_call(context.network_provider.proxy, 200000000, user, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, value=1 * 10 ** 18, abi = abi)
chain_sim.advance_blocks(1)

In [ ]:
wegld_mex_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
wegld_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[1]
mex_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[-3]

swaps_unencoded = [
    1,  # number of operations
    10 * 10 ** 18,  # amount for first operation
    1,  # number of swaps per first operation
    wegld_mex_contract.address, "swapTokensFixedInput", wegld_mex_contract.secondToken, 1,  # swap 1
]

args = [
    (wegld_mex_contract.secondToken, 0, 1),
    [
        wrap_egld_task(),
        smart_swap_task(swaps_unencoded),
    ]
]
endpoint_call(context.network_provider.proxy, 200000000, user, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, value=10 * 10 ** 18, abi = abi)
chain_sim.advance_blocks(1)

inconsistent swaps length

In [ ]:
wegld_mex_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
wegld_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[1]
mex_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[-3]

swaps_unencoded = [
    2,  # number of operations
    5 * 10 ** 17,  # amount for first operation
    1,  # number of swaps per first operation
    wegld_usdc_contract.address, "swapTokensFixedInput", wegld_usdc_contract.secondToken, 1,  # swap 1
    5 * 10 ** 17,  # amount for second operation
    1,  # number of swaps per second operation
    wegld_mex_contract.address, "swapTokensFixedInput", wegld_mex_contract.secondToken, 1,  # swap 1
    mex_usdc_contract.address, "swapTokensFixedInput", mex_usdc_contract.secondToken, 1,  # swap 2
]

args = [
    (wegld_usdc_contract.secondToken, 0, 1),
    [
        wrap_egld_task(),
        smart_swap_task(swaps_unencoded),
    ]
]
endpoint_call(context.network_provider.proxy, 200000000, user, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, value=1 * 10 ** 18, abi = abi)
chain_sim.advance_blocks(1)

Complete flow

In [ ]:
wegld_mex_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
wegld_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[1]
mex_usdc_contract: PairContract = context.get_contracts(config.PAIRS_V2)[-3]

swaps_unencoded = [
    2,  # number of operations
    5 * 10 ** 17,  # amount for first operation
    1,  # number of swaps per first operation
    wegld_usdc_contract.address, "swapTokensFixedInput", wegld_usdc_contract.secondToken, 1,  # swap 1
    5 * 10 ** 17,  # amount for second operation
    2,  # number of swaps per second operation
    wegld_mex_contract.address, "swapTokensFixedInput", wegld_mex_contract.secondToken, 1,  # swap 1
    mex_usdc_contract.address, "swapTokensFixedInput", mex_usdc_contract.secondToken, 1,  # swap 2
]

args = [
    ("EGLD", 0, 1),
    [
        wrap_egld_task(),
        smart_swap_task(swaps_unencoded),
        swap_input_task(wegld_usdc_contract.firstToken, 1),
        unwrap_egld_task()
    ]
]
endpoint_call(context.network_provider.proxy, 200000000, user, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, value=1 * 10 ** 18, abi = abi)

In [ ]:
from utils.utils_tx import multi_esdt_endpoint_call, ESDTToken
from utils.utils_chain import Account, WrapperAddress
from utils.utils_scenarios import get_token_in_account
from contracts.pair_contract import PairContract
from utils.contract_data_fetchers import PairContractDataFetcher
from multiversx_sdk.abi import TokenIdentifierValue, BigUIntValue

wegld_mex_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
pair_data_fetcher = PairContractDataFetcher(WrapperAddress(wegld_mex_contract.address), context.network_provider.proxy.url)

user = Account(pem_file=config.DEFAULT_ACCOUNTS)
user.address = WrapperAddress("erd1v4ms58e22zjcp08suzqgm9ajmumwxcy4hfkdc23gvynnegjdflmsj6gmaq")
user.sync_nonce(context.network_provider.proxy)

def find_lowest_input_with_same_output(starting_input, starting_token_id, resulting_token_id, pair_data_fetcher):
    
    token_eq = pair_data_fetcher.get_data("getEquivalent", [TokenIdentifierValue(resulting_token_id), BigUIntValue(1)])
    expected_output = pair_data_fetcher.get_data("getAmountOut", [TokenIdentifierValue(starting_token_id), BigUIntValue(starting_input)])

    lo = starting_input - 2 * token_eq
    hi = starting_input
    answer = None

    while lo <= hi:
        mid = (lo + hi) // 2
        out_amt = pair_data_fetcher.get_data(
            "getAmountOut", [TokenIdentifierValue(starting_token_id), BigUIntValue(mid)]
        )
        if out_amt == expected_output:
            answer = mid
            hi = mid - 1
        elif out_amt < expected_output:
            lo = mid + 1
        else:
            hi = mid - 1
    return answer

In [ ]:
swapped_token = wegld_mex_contract.secondToken
_, token_in_amount, _ = get_token_in_account(context.network_provider.proxy, user, swapped_token)
other_token = wegld_mex_contract.firstToken if swapped_token == wegld_mex_contract.secondToken else wegld_mex_contract.secondToken

if token_in_amount == 0:
    raise Exception("Not enough tokens in account")

token_in_amount = token_in_amount // 2 # swap half of the tokens

token_in_amount = find_lowest_input_with_same_output(token_in_amount, swapped_token, other_token, pair_data_fetcher)

token_out_amount = pair_data_fetcher.get_data("getAmountOut", [TokenIdentifierValue(swapped_token), BigUIntValue(token_in_amount)])
slippage = token_out_amount * 100 // 100
if slippage == 0:
    slippage = 1

swaps_unencoded = [
    1,  # number of operations
    token_in_amount,  # amount for first operation
    1,  # number of swaps per first operation
    wegld_mex_contract.address, "swapTokensFixedOutput", other_token, slippage,  # swap 1
]

swaps_unencoded_2 = [
    1,  # number of operations
    token_in_amount,  # amount for first operation
    2,  # number of swaps per first operation
    wegld_mex_contract.address, "swapTokensFixedOutput", other_token, slippage,  # swap 1
    wegld_mex_contract.address, "swapTokensFixedInput", swapped_token, 1,  # swap 2
]

halved_amount = find_lowest_input_with_same_output(token_in_amount//2, swapped_token, other_token, pair_data_fetcher)
halved_token_out_amount = pair_data_fetcher.get_data("getAmountOut", [TokenIdentifierValue(swapped_token), BigUIntValue(halved_amount)])

swaps_unencoded_3 = [
    2,  # number of operations
    halved_amount,  # amount for first operation
    1,  # number of swaps per first operation
    wegld_mex_contract.address, "swapTokensFixedOutput", other_token, halved_token_out_amount,  # swap 1
    token_in_amount - halved_amount,  # amount for second operation
    1,  # number of swaps per second operation
    wegld_mex_contract.address, "swapTokensFixedInput", other_token, 1,  # swap 1
]

swaps_unencoded_4 = [
    2,  # number of operations
    halved_amount,  # amount for first operation
    2,  # number of swaps per first operation
    wegld_mex_contract.address, "swapTokensFixedOutput", other_token, halved_token_out_amount,  # swap 1
    wegld_mex_contract.address, "swapTokensFixedInput", swapped_token, 1,  # swap 2
    token_in_amount - halved_amount,  # amount for second operation
    2,  # number of swaps per second operation
    wegld_mex_contract.address, "swapTokensFixedInput", other_token, 1,  # swap 1
    wegld_mex_contract.address, "swapTokensFixedInput", swapped_token, 1,  # swap 2
]

swaps_unencoded_5 = [
    2,  # number of operations
    halved_amount,  # amount for first operation
    2,  # number of swaps per first operation
    wegld_mex_contract.address, "swapTokensFixedInput", other_token, 1,  # swap 1
    wegld_mex_contract.address, "swapTokensFixedInput", swapped_token, 1,  # swap 2
    token_in_amount - halved_amount,  # amount for second operation
    2,  # number of swaps per second operation
    wegld_mex_contract.address, "swapTokensFixedInput", other_token, 1,  # swap 1
    wegld_mex_contract.address, "swapTokensFixedInput", swapped_token, 1,  # swap 2
]

def scen_args(index: int):
    scenarios_args = [
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (other_token, 0, 1),
            [
                smart_swap_task(swaps_unencoded),
            ]   # on 100 slippage, fails on mainnet, passes on upgraded; on lower slippage, passes on both
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (other_token, 0, 1),
            [
                swap_output_task(other_token, slippage),
            ]   # on 100 slippage, fails on mainnet, passes on upgraded; on lower slippage, passes on both
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (swapped_token, 0, 1),
            [
                swap_output_task(other_token, slippage),
                swap_input_task(swapped_token, 1),
            ]   # on 100 slippage, fails on mainnet, passes on upgraded; on lower slippage, passes on both
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            ("EGLD", 0, 1),
            [
                smart_swap_task(swaps_unencoded),
                unwrap_egld_task()
            ]   # on 100 slippage, fails on mainnet, passes on upgraded; on lower slippage, passes on both
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (swapped_token, 0, 1),
            [
                smart_swap_task(swaps_unencoded),
                swap_input_task(swapped_token, 1),
            ]   # on 100 slippage, fails on mainnet, passes on upgraded; on lower slippage, passes on both ?!??!?!?
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (swapped_token, 0, 1),
            [
                swap_input_task(other_token, 1),
                swap_input_task(swapped_token, 1)
            ]
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (swapped_token, 0, 1),
            [
                smart_swap_task(swaps_unencoded_2),
            ]
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (other_token, 0, 1),
            [
                smart_swap_task(swaps_unencoded_3),
            ]
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (swapped_token, 0, 1),
            [
                smart_swap_task(swaps_unencoded_4),
            ]
        ],
        [
            [ESDTToken(swapped_token, 0, token_in_amount)],
            (swapped_token, 0, 1),
            [
                smart_swap_task(swaps_unencoded_5),
            ]
        ]
    ]
    return scenarios_args[index]

args = scen_args(9)
multi_esdt_endpoint_call("", context.network_provider.proxy, 200000000, user, WrapperAddress(composable_tasks_contract.address), "composeTasks", args, abi = abi)
chain_sim.advance_blocks(1)

In [ ]:
chain_sim.advance_blocks(1)

In [ ]:
_, token_in_amount, _ = get_token_in_account(context.network_provider.proxy, user, swapped_token)
other_token = wegld_mex_contract.firstToken if swapped_token == wegld_mex_contract.secondToken else wegld_mex_contract.secondToken

if token_in_amount == 0:
    raise Exception("Not enough tokens in account")

token_in_amount = token_in_amount // 2 # swap half of the tokens

pair_data_fetcher = PairContractDataFetcher(WrapperAddress(wegld_mex_contract.address), context.network_provider.proxy.url)
token_out_amount = pair_data_fetcher.get_data("getAmountOut", [TokenIdentifierValue(swapped_token), BigUIntValue(token_in_amount)])
print(token_in_amount)
print(token_out_amount)

In [ ]:
print(token_in_amount)
token_out_amount = pair_data_fetcher.get_data("getEquivalent", [TokenIdentifierValue(swapped_token), BigUIntValue(token_in_amount)])
print(token_out_amount)
token_in_sim = pair_data_fetcher.get_data("getEquivalent", [TokenIdentifierValue(other_token), BigUIntValue(token_out_amount)])
print(token_in_sim)
token_out_amount = pair_data_fetcher.get_data("getEquivalent", [TokenIdentifierValue(swapped_token), BigUIntValue(token_in_sim+1)])
print(token_out_amount)

In [ ]:
token_in_sim = pair_data_fetcher.get_data("getEquivalent", [TokenIdentifierValue(other_token), BigUIntValue(1)])
print(token_in_sim)

In [ ]:
token_eq = pair_data_fetcher.get_data("getEquivalent", [TokenIdentifierValue(other_token), BigUIntValue(1)])
token_in_amount = token_in_amount - (token_in_amount % token_eq)
token_out_amount = pair_data_fetcher.get_data("getAmountOut", [TokenIdentifierValue(swapped_token), BigUIntValue(token_in_amount)])
print(token_in_amount)
print(token_out_amount)

Position creator

In [ ]:
swapped_token = wegld_mex_contract.secondToken
_, token_in_amount, _ = get_token_in_account(context.network_provider.proxy, user, swapped_token)
other_token = wegld_mex_contract.firstToken if swapped_token == wegld_mex_contract.secondToken else wegld_mex_contract.secondToken

if token_in_amount == 0:
    raise Exception("Not enough tokens in account")

token_in_amount = token_in_amount // 2 # swap half of the tokens

token_in_amount = find_lowest_input_with_same_output(token_in_amount, swapped_token, other_token, pair_data_fetcher)

token_out_amount = pair_data_fetcher.get_data("getAmountOut", [TokenIdentifierValue(swapped_token), BigUIntValue(token_in_amount)])
slippage = token_out_amount * 100 // 100
if slippage == 0:
    slippage = 1

args = [
    [ESDTToken(swapped_token, 0, token_in_amount)],
    wegld_mex_contract.address,
    1,
    1,
    wegld_mex_contract.address, "swapTokensFixedOutput", other_token, slippage,
    wegld_mex_contract.address, "swapTokensFixedInput", swapped_token, 1,
    wegld_mex_contract.address, "swapTokensFixedInput", other_token, 1
]

multi_esdt_endpoint_call("", context.network_provider.proxy, 200000000, user, WrapperAddress(position_creator_contract.address), "createLpPosFromSingleToken", args)
chain_sim.advance_blocks(1)